# Function Calling with LangChain Agents

In this notebook, we will leverage [Agents](https://python.langchain.com/docs/modules/agents/) in LangChain (particularly, custom-defined agents) to demonstrate the capabilities of function-calling-augmented LLMs. 

In [1]:
from langchain_community.chat_models import ChatOpenAI
from dotenv import load_dotenv
import os

load_dotenv()

True

In [2]:
llm = ChatOpenAI(
    openai_api_key=os.getenv("OPENAI_API_KEY"),
    temperature=0,
    model_name="gpt-3.5-turbo"
)

## 1. Calculator Function

Computers can solve complex math problems, yet if we ask GPT-3.5 (or GPT-4, for that matter) for the answer to `(4.5*2.1)^2.2`, it fails.

In [3]:
print(llm.invoke("(4.5*2.1)^2.2").content)

Calculating the expression (4.5*2.1)^2.2:

(4.5*2.1)^2.2 = (9.45)^2.2
(9.45)^2.2 ≈ 94.5^2.2
94.5^2.2 ≈ 1,048.52

Therefore, (4.5*2.1)^2.2 is approximately 1,048.52.


To address this problem, we can leverage function calling to allow the LLM to use an actual calculator tool.

### 1.1 Creating a Custom Tool

We will create a tool that will evaluate basic arithmetic math problems. There many ways to define tools - we enumerate some of these approaches below.

#### 1.1.1 OOP Approach

We can define our own tools with an inheritance-based approach. 

In [4]:
from langchain.tools import BaseTool

class CalculatorTool(BaseTool):
    # The name and description are required by LangChain - they are used to help the LLM figure out what tool to use
    name = "Calculator"
    description = "A simple calculator tool"
    
    def _run(self, input_expr: str) -> int:
        # The eval function is very dangerous, but for this example it's fine
        # It evaluates the string as a literal Python expression
        # e.g. eval("4.5*2") -> 9.0
        # but also eval("os.system('rm -rf /')") deletes everything
        # the two empty dictionaries can help to prevent some attacks. read more in the documentation
        return eval(input_expr, {}, {})

#### 1.1.2 Function Decorator Approach

Another method for custom tool definition is through the use of function [decorators](https://www.geeksforgeeks.org/decorators-in-python/) (a more advanced Python concept).

In [5]:
from langchain.tools import tool

@tool
def calculator(input_expr: str) -> int:
    """A simple calculator tool that evaluates a mathemtical expression and returns the result as an integer. Do not pass untrusted input."""
    return eval(input_expr, {}, {})

#### Defining an Agent

In [6]:
# when giving tools to LLM, we must pass as list of tools
# we will use the function tool
# the process would be the same for the OOP tool
# tools = [CalculatorTool]
tools = [calculator]

In [7]:
from langchain.agents import load_tools

tools = load_tools(
    ['llm-math'],
    llm=llm
)

In [8]:
from langchain.agents import initialize_agent

zero_shot_agent = initialize_agent(
    agent="zero-shot-react-description",
    tools=tools,
    llm=llm,
    verbose=True,
    max_iterations=3
)

zero_shot_agent("what is (4.5*2.1)^2.2?")



> Entering new AgentExecutor chain...
I need to calculate the result of (4.5*2.1) first before raising it to the power of 2.2.
Action: Calculator
Action Input: 4.5*2.1
Observation: Answer: 9.450000000000001
Thought:Now I need to raise 9.45 to the power of 2.2.
Action: Calculator
Action Input: 9.45^2.2
Observation: Answer: 139.9426129833306
Thought:I now know the final answer
Final Answer: 139.9426129833306

> Finished chain.


{'input': 'what is (4.5*2.1)^2.2?', 'output': '139.9426129833306'}

### 1.2 Using an Existing Tool

LangChain's [LLMMathChain](https://api.python.langchain.com/en/latest/chains/langchain.chains.llm_math.base.LLMMathChain.html) would be a good solution here. The LLMMathChain turns math problems into Python Code (using the LLM) and runs the code to obtain the result.

In [9]:
from langchain.chains import LLMMathChain
from langchain.agents import Tool

llm_math = LLMMathChain(llm=llm)

# initialize the math tool
math_tool = Tool(
    name='Calculator',
    func=llm_math.run,
    description='Useful for when you need to answer questions about math.'
)
# when giving tools to LLM, we must pass as list of tools
tools = [math_tool]

C:\Users\amira\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\langchain\chains\llm_math\base.py:57: UserWarning: Directly instantiating an LLMMathChain with an llm is deprecated. Please instantiate with llm_chain argument or using the from_llm class method.
  warnings.warn(


In [10]:
from langchain.agents import load_tools

tools = load_tools(
    ['llm-math'],
    llm=llm
)

In [11]:
from langchain.agents import initialize_agent

zero_shot_agent = initialize_agent(
    agent="zero-shot-react-description",
    tools=tools,
    llm=llm,
    verbose=True,
    max_iterations=3
)

zero_shot_agent("what is (4.5*2.1)^2.2?")



> Entering new AgentExecutor chain...
I need to calculate the result of (4.5*2.1) first before raising it to the power of 2.2.
Action: Calculator
Action Input: 4.5*2.1
Observation: Answer: 9.450000000000001
Thought:Now I need to raise 9.45 to the power of 2.2.
Action: Calculator
Action Input: 9.45^2.2
Observation: Answer: 139.9426129833306
Thought:I now know the final answer
Final Answer: 139.9426129833306

> Finished chain.


{'input': 'what is (4.5*2.1)^2.2?', 'output': '139.9426129833306'}